### Medium 사이트 크롤링

블로깅 사이트인 medium의 업데이트 사항을 추적해서 telegram으로 인포를 주는 스크립트를 짜보도록 하겠습니다.

스크립트 워크 플로우를 글로 적어본다면 다음과 같습니다.

1. request로 medium 사이트 url로 웹페이지를 요청한다.
2. 응답한 내용의 텍스트(html)을 beautifulsoup로 넣고 파싱한다.
3. 파싱된 내용중 제목만을 추출한 리스트를 확보한다.
4. 기존에 저장한 리스트와 비교하여 새로 업데이트 된 사항이 있는지 확인
5. 업데이트 된 사항이 있으면 해당 제목의 url을 추출한다.
6. 업데이트된 url을 telegram으로 전송한다.

지난 (9월 28일)에 진행하였던  4.telegram 으로 메시지 보내기와 3. 파일 입출력 두가지를 응용한 실제 예제입니다.

medium의 url구조를 보면 https://medium.com/topic/technology 과 같이 topic 또는 https://medium.com/@caraty_60730/latest 그리고 https://medium.com/tag/bitcoin 와 같이 @id  또는 tag 로 시작하는 구조를 가지고 있습니다.

응답받은 결과의 구조가 비슷하니 "https://medium.com/tag/bitcoin/latest" 을 기준으로 시작해보겠습니다.


In [1]:
# 먼저 라이브러리를 import 합니다.
import requests
from bs4 import BeautifulSoup as bs

In [2]:
url_tag = "https://medium.com/tag/bitcoin/latest"
resp = requests.get(url_tag)

In [3]:
resp #200! 잘 왔습니다. 

<Response [200]>

In [4]:
len(resp.text) #텍스트를 열어보지는 않았지만 글자수가 적당한걸로 봐서 잘온거 같습니다.

189371

In [5]:
bs_html = bs(resp.text, "html.parser") #html text 를 뷰티풀수프에 넣습니다.

In [6]:
post_list = bs_html.find_all('h3') # 제목 테그들이 모두 h3 를 달고 있더군요.
post_list

[<h3 class="graf graf--h3 graf--leading graf--title" id="6e7c" name="6e7c">The ICO Summit: The Bitcoin industry is thriving</h3>,
 <h3 class="graf graf--h3 graf--leading graf--title" id="b1b0" name="b1b0">Trackr is now live on HitBTC for trading!</h3>,
 <h3 class="graf graf--h3 graf--leading graf--title" id="f732" name="f732">Circulating Supply of Dentacoin</h3>,
 <h3 class="graf graf--h3 graf-after--figure graf--trailing graf--title" id="0c22" name="0c22"><strong class="markup--strong markup--h3-strong">The Future of Bitcoin and Other Cryptocurrencies</strong></h3>,
 <h3 class="graf graf--h3 graf--leading graf--title" id="b667" name="b667">Cloud Application Exchange Bernama “WIRELINE”</h3>,
 <h3 class="graf graf--h3 graf--leading graf--title" id="def7" name="def7">Minerva - Децентрализованные умные деньги в ETHEREUM Blockchain</h3>,
 <h3 class="graf graf--h3 graf--leading graf--title" id="b3e1" name="b3e1"><strong class="markup--strong markup--h3-strong">How to get VariabL Contributio

### medium html 구조

div 태그란, 하나 이상의 태그들을 묶어서 스타일을 지정할 때 사용하는 것으로 웹페이지의 영역을 나누어서 같은 배경이나 컬러를 지정할때 사용합니다.

<img src="./images/medium_1.png">개발자도구로 본 medium 포스트 </img>

하나이 포스트를 나타내는 글상자는 '<div class="js-tagStream">'아래에 또 다른 div로 둘러쌓여진 글상자가 이어져 있습니다.

이런! 몇개의 글을은 h3-제목 태그를 가지고 있지 않습니다!<br>
<img src='./images/medium_2.png'>None h3 tag</img><br>
조금더 공통적인 부분을 찾아봅니다.<br> streamItem streamItem--postPreview js-streamItem 클래스로 나뉘어져 있는 걸로 리스트를 뽑아봅니다.

In [7]:
post_list = bs_html.find_all('div', {'class':'streamItem streamItem--postPreview js-streamItem'})
len(post_list)

10

In [8]:
#enumerate 를 사용해 봅니다.
test_list = ['a', 'b', 'c']
for idx, char in enumerate(test_list):
    print(idx, char) # enumerate를 씌워주면 순서 번호와 같이 두개의 변수를 꺼내옵니다.

0 a
1 b
2 c


In [9]:
#모두 다 있는지 확인해 봅니다.
for idx, c in enumerate(post_list):
    print(idx, '======\n'+ str(c))
    """0 =======
    html내용들..
       1 =======
    html내용들..
       2 ....
       
    """

0 ======
<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackedPost" data-post-id="48872817b0b0" data-source="---------0----------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link avatar u-baseColor--link" data-action="show-user-card" data-action-type="hover" data-action-value="133fe01ba1e5" data-user-id="133fe01ba1e5" dir="auto" href="https://medium.com/@orbe_"><img alt="Go to the profile of Or-Be" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/1*pExK5UIo-PR6S27Tk-qznw.jpeg"/></a></div><div class="postMetaInline postMetaInline-authorLockup u-flex1 u-noWrapWithEllipsis"><a class="link link link--darken link--accent u-acce

href 속성은 hyperlink(다른 웹페에지의 링크)를 포함하고 있으므로 href를 가지고 있는 태
그를 찾아봅시다. 아쉽게도 공통적인 링크는 없어보입니다.

하이퍼링크만 있는 게시글이나 medium에서 올린 게시글이 각기 다른 형태를 띄고 있으므로
조건문을 통해서 링크들을 추출해줘야 겠습니다.

In [10]:
post_2_link = post_list[2].find('div', {'class':'postArticle-readMore'})
post_2_link # 중간정도에 https:로 시작하는 링크가 보입니다. 여기서 
#<a> ~</a>태그로 둘러 쌓여있는 내에 속성들중에 하나인 data-action-value로 있습니다.

<div class="postArticle-readMore"><a class="button button--smaller button--chromeless u-baseColor--buttonNormal" data-action="open-post" data-action-source="---------2----------------" data-action-value="https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------" data-post-id="851f298836bb" href="https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------">Read more…</a></div>

In [11]:
post_2_link.a.get('data-action-value') # 드디어 찾았습니다.
#여기서 ?source부분은 필요 없으므로(혹은 페이지가 바뀌면서 숫자가 바뀔수 있으므로) 빼봅시다

'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------'

In [12]:
str(post_2_link.a.get('data-action-value')) #슬라이스 하여줍니다. 어디까지?!

'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------'

In [13]:
#string 객체의 find로 어디에 위치하고 있는디 확인합니다.
str(post_2_link.a.get('data-action-value')).find('?source=') 

90

In [14]:
str(post_2_link.a.get('data-action-value'))[:114] #

'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2------'

위의 과정을 함수로 만들어 보겠습니다.<br> 지저분한 url을 정리하는 함수

In [15]:
def url_trim(url_string):
    pos = str(url_string).find('?source=')
    return str(url_string)[:pos]

In [16]:
url_trim(post_2_link.a.get('data-action-value')) # 잘 나오는군요

'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb'

In [17]:
len(resp.text)

189371

In [18]:
import re 

In [19]:
# 정규식 모듈로도 더 빠르게 할 수 있습니다.
re_result = re.findall(r'data-action-value="(http.+?)"', resp.text)
re_result

['https://medium.com/@orbe_/the-ico-summit-the-bitcoin-industry-is-thriving-48872817b0b0?source=---------0----------------',
 'https://medium.com/@orbe_/the-ico-summit-the-bitcoin-industry-is-thriving-48872817b0b0?source=---------0----------------',
 'https://medium.com/@trackr.im/trackr-is-now-live-on-hitbtc-for-trading-bbce3eacf0c9?source=---------1----------------',
 'https://hitbtc.com/exchange/TKR-to-ETH',
 'https://etherdelta.com/#TKR-ETH',
 'https://medium.com/@trackr.im/trackr-is-now-live-on-hitbtc-for-trading-bbce3eacf0c9?source=---------1----------------',
 'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------',
 'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------',
 'https://medium.com/@dentacoin/circulating-supply-of-dentacoin-ffcca3c72a04?source=---------3----------------',
 'https://medium.com/@dentacoin/circulating-supply-of-de

In [20]:
new_post_list = list(set(re_result))
new_post_list

['https://medium.com/@trackr.im/trackr-is-now-live-on-hitbtc-for-trading-bbce3eacf0c9?source=---------1----------------',
 'https://medium.com/@hongalex/the-future-of-bitcoin-and-other-cryptocurrencies-abafbc563d43?source=---------4----------------',
 'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb?source=---------2----------------',
 'https://medium.com/koles-coin-news/wall-street-traders-are-turning-to-cryptocurrencies-c94a97cd4db4?source=---------5----------------',
 'https://hitbtc.com/exchange/TKR-to-ETH',
 'https://medium.com/@orbe_/the-ico-summit-the-bitcoin-industry-is-thriving-48872817b0b0?source=---------0----------------',
 'https://medium.com/@Jeffery1st/martin-green-on-bitcoin-78e37859f3aa?source=---------9----------------',
 'https://medium.com/@tirta16/cloud-application-exchange-bernama-wireline-3345c7ada21?source=---------6----------------',
 'https://blog.variabl.io/how-to-get-variabl-contribution-tokens-vct-a-brief-practical

beautifulsoup으로 파싱하여 태그들을 따라가서 추출하는 방법과,<br> 파이썬 정규식을 활용하여 페이지내의 규칙을 찾아 적용하여 한것과 후자 빠지지 않고 추출되는것 같습니다.<br>
시간 여건상 정규식을 활용하여 url 들을 추출하고 파일에 기록해봅시다.

In [21]:
#medium post의 경우는 ?source 가 붙으므로 이전에 만들었던 함수로 제거합니다.
update_posts = []
for i in new_post_list:
    update_posts.append(url_trim(i))

In [22]:
update_posts

['https://medium.com/@trackr.im/trackr-is-now-live-on-hitbtc-for-trading-bbce3eacf0c9',
 'https://medium.com/@hongalex/the-future-of-bitcoin-and-other-cryptocurrencies-abafbc563d43',
 'https://medium.com/koles-coin-news/israeli-governments-announcement-about-ico-851f298836bb',
 'https://medium.com/koles-coin-news/wall-street-traders-are-turning-to-cryptocurrencies-c94a97cd4db4',
 'https://hitbtc.com/exchange/TKR-to-ET',
 'https://medium.com/@orbe_/the-ico-summit-the-bitcoin-industry-is-thriving-48872817b0b0',
 'https://medium.com/@Jeffery1st/martin-green-on-bitcoin-78e37859f3aa',
 'https://medium.com/@tirta16/cloud-application-exchange-bernama-wireline-3345c7ada21',
 'https://blog.variabl.io/how-to-get-variabl-contribution-tokens-vct-a-brief-practical-guide-7493804ecc00',
 'https://medium.com/@dentacoin/circulating-supply-of-dentacoin-ffcca3c72a04',
 'https://etherdelta.com/#TKR-ET',
 'https://medium.com/@tirta16/minerva-%D0%B4%D0%B5%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D0%B8%D0%B

이제 처음 페이지를 로딩했을떄의 url들을 확보하였으니 하나씩 파일에 써주도록 하겠습니다.

In [23]:
for p in update_posts:
    with open('./medium_post.txt', 'a', encoding='utf8') as f:
        f.write(p+'\n') #\n 다음줄을 구분자로 쓸것입니다.

In [24]:
#기존의 저장된 post_url의 링크를 old_post_list로 불러와서 기존것인지 새것인지 구별해 봅시다
with open('./medium_post.txt', 'r', encoding='utf8') as f:
    post_text = f.read()

old_post_list = post_text.split('\n') #  구분자가 \n이라고 말씀 드렸죠

In [25]:
#첫번째 부분을 살짝 변형해서 있는건지 없는 건지 확인
po_url = 'https://medium.com/@zycrypto/south-african-reserve-bank-says-bitcoin-not-a-legal-tender-makes-move-for-regulation-7178949dd9e'

In [26]:
po_url in old_post_list

False

In [27]:
po_url+'3' in old_post_list

True

이처럼 url이 새로운 것이 있다면 False 기존 url 이라면 True로 판단합니다.<br> 
이제는 무한루프를 돌면서 새로운 포스팅이 뜰때 print해주는 스크립트를 써봅니다.

In [ ]:
#위에서 사용했던 코드를 가져옵니다.

status = True
while status:
    resp = requests.get(url_tag)
    re_result = re.findall(r'data-action-value="(http.+?)"', resp.text)
    new_post_list = list(set(re_result))
    
    # 새로운 post url들을 정리해서 update_posts에 넣습니다.
    update_posts = []
    for i in new_post_list:
        update_posts.append(url_trim(i))
    
    #기존의 url text를 불러옵니다.
    with open('./medium_post.txt', 'r', encoding='utf8') as f:
        post_text = f.read()
        
    old_post_list = post_text.split('\n')
    
    for post in update_posts:
        if post in old_post_list:
            print("Exist post")
            pass # True 로 리턴되는 경우는 pass 입니다. 왜냐하면 기존의 post 이니까요
        else:
            print("New Post Found!!")
            #새로운 post를 씁니다.
            with open('./medium_post.txt', 'a', encoding='utf8') as f:
                f.write(post+'\n')
                
            print(">> {} letter write on medium_post.txt".format(len(post)))
            ststus = False